In [ ]:
%config Completer.use_jedi = False

In [ ]:
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mne
import json
import scipy.stats
import scipy.signal as sig

# import scipy.io as sio
# from scipy import signal

import pac

import simple_pipeline


suffix = '_1_200'#'_1ch_nv'
gamma = [ 1, 200]
beta  = [ 1, 50]

filt_wind = np.ones((3, 3))
filt_wind /= filt_wind.sum()

# functions

In [ ]:
with open('config.json') as f:
    config = json.load(f)
    channels = config['channels']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
linestyles = ['-', ':', '--']

groups = ['PD Med Off', 'PD Med On', 'CTL']
event_types = ['Target', 'Standard', 'Novelty']

In [ ]:
def plot_pac(pac, high_freq=gamma, low_freq=beta, ax=None, interpolation='None', **kwargs):
    if ax is None:
        fig = plt.figure(figsize=(7, 15))
        ax = fig.subplots()

    im = ax.imshow((pac), origin='lower', interpolation=interpolation, #'nearest', 
                   extent=low_freq+high_freq,
#                    aspect='auto', )
                   aspect=np.diff(low_freq)/np.diff(high_freq), **kwargs)

    if ax is None:
        plt.show()
        
    return im

In [ ]:
def get_percent(arr, thr=0.95):
    if arr.ndim > 1:
        arr = arr.ravel()
    freq, bins = np.histogram(arr, bins=100)
    return (
        bins[:-1][(freq / freq.sum()).cumsum() > thr][0], 
        bins[1:][(freq / freq.sum()).cumsum() > thr][0]
    )

In [ ]:
def save_fig(path):
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    plt.savefig(path)

# Create Task list in `tasks_df`

In [ ]:
if __name__ == '__main__':
    tasks_df = simple_pipeline.create_tasks_df()
    
    completed = []
    for task in tasks_df.iloc:
        json_path = os.path.join(task['dir'], task['file_formatter'].format(f'completed{suffix}.json'))
        completed.append(os.path.exists(json_path))

    tasks_df = pd.concat([tasks_df, pd.DataFrame({'completed': completed})], axis=1)

In [ ]:
os.path.join(task['dir'], task['file_formatter'].format(f'mvls{suffix}.npz'))

# Load Data

In [ ]:
# MVL

mvl_2ds_time = [[] for k in groups] # np.zeros((3, 64, 169, 37 * 12))
mvl_2ds = [[] for k in groups] # np.zeros((3, 64, 61, 13 )//169, 37))
mvls = [[] for k in groups] # np.zeros((3, 64))

for task in tasks_df.iloc:
# if 1:
    task_mvls = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvls{suffix}.npz')))
    task_mvl_2ds = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2ds{suffix}.npz')))
    
    # mvls
    nbchan = task_mvls[task_mvls.files[0]].shape[0]
    mvl = np.zeros((3, nbchan))
    for i, event_type in enumerate(sorted(task_mvls.files)):
        mvl[i] = task_mvls[event_type].diagonal()
        
    mvls[task.pd_drug_type].append(mvl)
    
    # mvl_2ds
    mvl_2d = np.zeros((3, nbchan, gamma[1] - gamma[0] + 1, beta[1] - beta[0] + 1))
    for i, event_type in enumerate(sorted(task_mvl_2ds.files)):
        mvl_2d[i] = task_mvl_2ds[event_type].diagonal(0, 0, 1).transpose((2, 0, 1))
        
    mvl_2ds[task.pd_drug_type].append(mvl_2d)

mvls = np.array(mvls)              # --> (pd_drug_type, subjects, event_types, channels)
mvl_2ds = np.array(mvl_2ds)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs)
mvl_2ds_time = np.array(mvl_2ds_time)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs * 12)

In [ ]:
with open('config.json') as f:
    config = json.load(f)
base_path = config['BASE_PATH']
ds_path = os.path.join(base_path, 'ds003490-download')
nbchan = len(config['channels']) - 1

In [ ]:
# MVL 2Ds

mvl_2ds_mmap_path = os.path.join(ds_path, f'MVL_2ds_{suffix}.mmap')

mvl_2ds = np.memmap(mvl_2ds_mmap_path, dtype=float, mode='w+',
                    shape=(3,  # pd_drug_type
                           25, # Subjects
                           3,  # event_types
                           nbchan, # Channels
                           gamma[1] - gamma[0] + 1, # High Freq
                           beta[1] - beta[0] + 1, # Low Freq
                          ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_mvl_2ds = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2ds{suffix}.npz')))
    for i, event_type in enumerate(sorted(task_mvl_2ds.files)):
        mvl_2ds[task.pd_drug_type, subs[task.pd_drug_type], i] = \
            task_mvl_2ds[event_type].diagonal(0, 0, 1).transpose((2, 0, 1))
    
        
    subs[task.pd_drug_type] += 1

# mvl_2ds = np.array(mvl_2ds)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs)
# mvl_2ds_time = np.array(mvl_2ds_time)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs * 12)

In [ ]:
mvl_time_mmap_path = os.path.join(ds_path, f'PAC_time_{suffix}.mmap')
mvl_time = np.memmap(mvl_time_mmap_path, dtype=float, mode='w+',
                     shape=(3,  # pd_drug_type
                            25, # Subjects
                            3,  # event_types
                            nbchan, # Channels
                            gamma[1] - gamma[0] + 1, # High Freq
                            beta[1] - beta[0] + 1, # Low Freq
                            6, # Time sections
                           ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_mvl_2ds_time = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2d_times{suffix}.npz')))
    
    # mvl_2ds
    for i, event_type in enumerate(sorted(task_mvl_2ds_time.files)):
        mvl_time[task.pd_drug_type, subs[task.pd_drug_type], i] = \
            task_mvl_2ds_time[event_type]
    
#         mvl_2d_time[i] = task_mvl_2ds_time[event_type]
    subs[task.pd_drug_type] += 1
        
#     mvl_2ds_time[task.pd_drug_type].append(mvl_2d_time)

In [ ]:
# ERP

nbtime = 601

erp_mmap_path = os.path.join(ds_path, f'ERPs_{suffix}.mmap')
epochs = np.memmap(erp_mmap_path, dtype=float, mode='w+',
                     shape=(3,  # pd_drug_type
                            25, # Subjects
                            3,  # event_types
                            2,  # (mean, std)
                            nbchan, # Channels
                            nbtime, # time
                           ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_epochs = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'epochs.npz')))
    
    # epochs
    for i, event_type in enumerate(sorted(task_epochs.files)):
        print(subs, task.pd_drug_type)
        epochs[task.pd_drug_type, subs[task.pd_drug_type], i, 0] = task_epochs[event_type].mean(axis=0)
        epochs[task.pd_drug_type, subs[task.pd_drug_type], i, 1] = task_epochs[event_type].std(axis=0)
        
    # epochs[task.pd_drug_type].append(epoch)
    subs[task.pd_drug_type] += 1

In [ ]:
[f for f in os.listdir(ds_path) if f.endswith('mmap')]

In [ ]:
!ls -sh /home/kiani/DS/ds003490-download/*.mmap